<a href="https://colab.research.google.com/github/arianza1210/Data_Scientist/blob/main/time_series_analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import datetime
from datetime import date
import prophet
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
from scipy.signal import savgol_filter


In [7]:

fn='/content/Daylight_saving_time.csv'

In [10]:
#read data 
filepath=os.path.join(os.path.dirname(os.path.realpath('__file__')), fn)
df=pd.read_csv(filepath)
#ubah nilai 0  menjadi Nan
df=df.replace('0',np.nan)
#format datestamp
df.ds=pd.to_datetime(df.ds)
df.yOri = df.y
df.y = np.log(df.y)

<ipython-input-10-48b02e70a486>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.yOri = df.y
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [13]:
#spesial events

springforward = pd.DataFrame({
    'holiday':'springforward',
    'ds': pd.to_datetime(['2008-03-09','2009-03-08',
                          '2010-03-14','2011-03-13',
                          '2012-03-11','2013-03-10',
                          '2014-03-09','2015-03-08']),
    'lower_window':-14,
    'upper_window':14,
})

fallback = pd.DataFrame({
    'holiday':'fallback',
    'ds': pd.to_datetime(['2008-11-02','2009-11-01',
                          '2010-11-07','2011-11-06',
                          '2012-11-03','2013-11-03',
                          '2014-11-02','2015-11-01']),
    'lower_window':-14,
    'upper_window':0,
})


news = pd.DataFrame({
    'holiday':'news',
    'ds': pd.to_datetime(['2010-04-14','2010-08-12',
                          '2011-02-08','2011-06-15',
                          '2011-09-15','2013-09-20',
                          '2012-11-05','2015-07-08',
                          '2014-07-22','2015-10-25']),
    'lower_window':-7,
    'upper_window':21,
})

holidays = pd.concat((springforward,fallback,news))
holidays


,holiday,ds,lower_window,upper_window
0,springforward,2008-03-09,-14,14
1,springforward,2009-03-08,-14,14
2,springforward,2010-03-14,-14,14
3,springforward,2011-03-13,-14,14
4,springforward,2012-03-11,-14,14
5,springforward,2013-03-10,-14,14
6,springforward,2014-03-09,-14,14
7,springforward,2015-03-08,-14,14
0,fallback,2008-11-02,-14,0
1,fallback,2009-11-01,-14,0


In [19]:
H=365
h=H/2

result =pd.DataFrame()
cutoff=[]

i=0
while(len(df)-i > 3*H):
  train = df[i:(i+(3*H))]

  m= prophet(interval_width=0.95,
                changepoint_prior_scale=0.01, # default is 0.05, decreasing it makes trend more generalizable
                holidays=holidays,
                holidays_prior_scale=10, # default is 10, strength of the holiday components
                yearly_seasonality=True, # since daylight savings occur annually
                weekly_seasonality=True, # people conduct more searches on weekdays
                seasonality_prior_scale=10,)
  m.fit(train);
  future=m.make_future_dataframe(periods=H)

  forecast=m.predict(future)
  resultsH=forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(H)
  resultsH=df.merge(resultsH,how='right')
  resultsH = resultsH.sort_values(by='ds')
    
    # record cutoff dates
  cutoffDate = resultsH['ds'].iloc[0]
  cutoffDate = cutoffDate.strftime('%Y %b')
  cutoff = cutoff + [cutoffDate]
    
    # compile results
  results = pd.concat((results, resultsH))
    
  print ('Counting the days...', i)
  i = i + h



TypeError: ignored